Zaynab Batool Reza

i180419

i180419@nu.edu.pk

#Code Implementation

## **Importing Libraries**


In [2]:
import pandas as pd
import numpy as np
import spacy as sp

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Reading Corpus

In [37]:
def readFile(filename):
    N=500
    text=""
    with open('/content/drive/MyDrive/'+filename,'r') as file: 
        text=file.read()
    text=text.replace('\n', ' ') #replace end of line characters
    text= (text.split()) #divide the text into words

    count=0
    
    words=[]  
    for w in text:
        words.append(w)
        count=count+1 #count total number of words
    print(count)
    return words
corpus=readFile('data.txt') #corpus saved in a list that has all the words in the text. Will be used in later functions 

63190876


### Checking To Ensure Entire File Has Been Read
Which it has if lenth of corpus is same as the count printed by previous cell

In [38]:
corpus[-2:len(corpus)]
print(len(corpus))

63190876


## Training UniGram using Corpus
Each key in the dictionary represents a unique word and the value against it is the number of times it appears in the corpus provided divided by total words in the corpus, thus representing the word's probability

In [39]:
def generateUnigram(words):
    count={}
    unigram={}
    for w in words:
        if w in count:
            count[w]+=1
        else:
            count[w]=1
            
    for key in count:        
        unigram[key]=count[key]/float(len(words))
    return unigram

unigram=(generateUnigram(corpus)) #unigram stored

print(unigram['sab']) #checking


0.0033060785547584432


In [7]:
print(len(unigram))

35834


## Creating Dictionary from Misspellings File
Each key in the dictionary is a correctly spelled word and against it as values is a list of the misspellings of that word as provided in file

In [8]:
def createMisspellingsDict(filename):
    file=open('/content/drive/MyDrive/'+filename,'r') #open in read mode
    lines=file.readlines()
    misspellings={}
    for line in lines:
        #print("Line{}: {}".format(1, line.strip()))
        fullLine=line.strip()
        correctWrong=fullLine.split(',')
        correct=correctWrong[0]
        tempwrongs=correctWrong[1].split(' ')
        wrongs=tempwrongs[1].split('\t')
        if correct not in misspellings:
            misspellings[correct]=[]
        misspellings[correct]=wrongs
    misspellings.pop('Correct')
    return misspellings

misspellings=createMisspellingsDict('misspellings.txt') #misspellings dictionary saved
print(misspellings['gujrati']) #checking


['gujriti', 'gujratij', 'gujraoti', 'gujrsati', 'gujoati', 'gujrasti', 'gujeati']


## Find Operation Peformed Between Words and Involved Letters
### Assuming edit distance of 1
The function takes the correct and wrong word, if wrong word is longer than correct it marks it as insertion and finds the letter inserted and the one before.
If wrong word is smaller then operation is considered delete and involved letters are found.
If both words are the same length, then the differing letter is found. If the next letter in the wrong word is the current one in correct and current one in wrong is next letter in correct word then it is marked as transpose, else it is considered substitution

In [9]:
def getOperationLetters(correct, wrong):
    x=""
    y=""
    operation=""
    letterFound=False
    if len(correct) < len(wrong): #insertion has taken place
        operation='insert'
        for i, letter in enumerate(correct):
            if (letter != wrong[i]):
                if i==0:
                    x='#'
                    y=wrong[i]
                else:
                    x=correct[i-1]
                    y=wrong[i]
                letterFound=True
                break
        if (letterFound==False):
            x=correct[-1]
            y=wrong[-1]
    elif len(correct) > len(wrong): #deletion has taken place
        operation='delete'
        for i, letter in enumerate(wrong):
            if (letter != correct[i]):
                if i==0:
                    x='#'
                    y=correct[i]
                else:
                    x=correct[i-1]
                    y=correct[i] 
                letterFound=True
                break
            if (letterFound==False):
                x=correct[-2]
                y=correct[-1]
                
    elif len(correct)==len(wrong): #substition or transpose has taken place
        for i, letter in enumerate(correct):
            if (letter != wrong[i]):
                if (i<len(wrong)-1):
                    if letter==wrong[i+1] and correct[i+1]==wrong[i]: #transpose
                        operation='transpose'
                        x=letter
                        y=correct[i+1]
                        break
                    else: #substitution
                        operation='substitute'
                        x=wrong[i]
                        y=letter
                        break
                else:
                    operation='substitute'
                    x=wrong[i]
                    y=letter
                    break
                    
                    
                    
    return operation,x,y

In [10]:
#Checking the function
op, x, y=getOperationLetters('woh', 'oh')
print("Operation = ", op, " x= ", x, " y= ", y)
op, x, y=getOperationLetters('ko', 'kwo')
print("Operation = ", op, " x= ", x, " y= ", y)
op, x, y=getOperationLetters('usary', 'usaey')
print("Operation = ", op, " x= ", x, " y= ", y)
op, x, y=getOperationLetters('usay', 'suay')
print("Operation = ", op, " x= ", x, " y= ", y)
op, x, y=getOperationLetters('ka', 'kaz')
print("Operation = ", op, " x= ", x, " y= ", y)
op, x, y=getOperationLetters('ka', 'k')
print("Operation = ", op, " x= ", x, " y= ", y)
op, x, y=getOperationLetters('pichlaa', 'pichla')
print("Operation = ", op, " x= ", x, " y= ", y)

Operation =  delete  x=  #  y=  w
Operation =  insert  x=  k  y=  w
Operation =  substitute  x=  e  y=  r
Operation =  transpose  x=  u  y=  s
Operation =  insert  x=  a  y=  z
Operation =  delete  x=  k  y=  a
Operation =  delete  x=  a  y=  a


## Generating Error Model Edit Tables
The dictionary for misspellings is traversed and for each correct and wrong word pair, the getOperationLetters function is called to find what operation is done and what letters are involved. The respective table is updated accordingly, where each table is a DataFrame and at the end all tables are returned

In [13]:
def generateEditTables(misspellings):
    alphabet=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    sub=pd.DataFrame(0,columns=alphabet, index=alphabet)
    rev=pd.DataFrame(0,columns=alphabet, index=alphabet)
    alphabetStart=['#','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    delete=pd.DataFrame(0,columns=alphabet, index=alphabetStart)
    insert=pd.DataFrame(0,columns=alphabet, index=alphabetStart)

    #for insert table, y is the letter in wrong word inserted after x
    #for delete table, y is the letter that has been deleted after x
    #for substitute table, y is the letter in correct word that is substituted by x
    #for reverse table, xy are the letters in correct word that have been reversed to become yx in wrong word
    
    for correct in misspellings:
        for wrong in misspellings[correct]:
            op, x, y=getOperationLetters(correct, wrong)
            if (x=="" or y==""):
                print("Correct word: ", correct, "Wrong Word: ", wrong)
            if (op=='insert'):
                insert[y][x]+=1 #in column y at index x
            elif (op=='delete'):
                delete[y][x]+=1
            elif (op=='transpose'):
                rev[y][x]+=1
            elif (op=='substitute'):
                sub[y][x]+=1
    
    return sub, rev, delete, insert


sub, rev, delete, insert=generateEditTables(misspellings) #sub, rev, delete and insert tables saved
insert #checking



,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
#,594,650,705,682,710,707,683,655,707,671,680,694,677,662,671,679,744,713,691,718,730,732,710,771,681,744
a,548,693,644,652,662,641,656,608,577,629,628,616,638,592,679,656,653,507,606,620,670,633,626,656,634,648
b,57,81,95,97,60,86,66,81,91,83,72,72,85,69,73,93,95,85,97,96,98,90,100,97,81,88
c,55,64,69,66,58,67,68,31,58,63,73,75,78,67,63,92,62,64,70,76,60,61,59,79,69,55
d,99,109,127,136,97,148,114,111,98,130,109,113,131,126,116,128,126,120,131,125,126,131,115,130,131,149
e,273,298,248,230,235,272,270,267,267,277,244,290,261,229,256,266,254,219,294,236,277,286,259,243,241,270
f,23,39,58,38,32,44,38,37,32,41,47,35,54,33,40,37,43,35,36,40,34,43,39,28,39,31
g,56,88,88,80,74,74,74,58,80,91,71,86,88,68,79,84,80,57,69,85,72,70,87,65,84,81
h,112,206,210,237,185,228,234,217,192,228,229,225,240,212,185,225,249,226,222,219,222,222,232,215,224,218
i,229,254,259,252,250,246,248,261,244,288,262,264,257,235,269,236,243,248,220,240,276,275,278,242,266,254


## Generating Character Level Bigram Model (data)
Every word in the corpus is broken into character level tuples of 2. Then the dictionary is updated where each key is a tuple and its value is the number of times it appears in the text

In [14]:
def generateBigram(corpus):
    tuples=[]
    count={}
    for word in corpus:
        cword='#'+word #adding the starting symbol to bigram model
        for i in range(len(cword)-1):
            l1=cword[i]
            l2=cword[i+1]
            t=tuple((l1,l2))
    # for t in tuples:
            if t in count:
                count[t]+=1
            else:
                count[t]=1
    return count
bigram=generateBigram(corpus) #bigram for data.txt using the corpus where it was stored

## Calculate P(x|w) Where w Is a Candidate Word

### Get Count of A Letter in Data
Every word in the corpus is broken into characters. Then the dictionary is updated where each key is a letter and its value is the number of times it appears in the text.

In [15]:
def getCountUnigram(corpus):
  countunigram={}
  for word in corpus:
    for letter in word:
      if letter in countunigram:
        countunigram[letter]+=1
      else:
          countunigram[letter]=1

  return countunigram

countunigram=getCountUnigram(corpus) #countunigram for data.txt using the corpus where it was stored
print(countunigram['y']) #checking

6922251


### Get Probability
Using the getOperationLetters, the function gets what operation is performed between the words and upon which letters. If insert or subsitute then the respective value in the edit table is divided by number of times the correct letter appears in the text, in insert table this letter is x and in substitute this letter is y.
If delete or tranpose then the value in the edit table is divided by the number of times the two letters appear consecutively in the text using the bigram generated before.

In [21]:
def getProbability(typo, candidate):
    op, x, y=getOperationLetters(candidate, typo)
    if (op=='insert'):
        #print("inserted ", y, "after",x)
        if x!='#':
            p=(insert[y][x])/float(countunigram[x]) #y is inserted after x
        else:
            p=(insert[y][x])/float(len(corpus))
    elif (op=='delete'):
        p=(delete[y][x])/float(bigram[(x,y)]) #y is deleted after x
    elif (op=='transpose'):
        p=(rev[y][x])/float(bigram[(x,y)]) #reversal of xy
    elif (op=='substitute'):
        p=(sub[y][x])/float(countunigram[y]) #x is substituted by y
    return p

print(getProbability('nhi', 'nahi')) #checking
    

5.062535475717346e-06


## Get Edit Distance Between Two Words x,w
This function gets the levenshtein edit distance between two words. Since transposition is considered 2 occurences of substitution, if transposition occurs in the word, the edit distance is subtracted by 1.
This subtraction is only done once since we will be considering those words with only 1 edit distance later on anyway. So even if there were multiple transpositions and decrementing it by 1 wont give accurate edit distance, it will not matter as edit distance will have exceeded 1 already and we will be discarding that word as a candidate as instructed by the assignment document.
We only need to cater to words where transposition occurs once and that has been done.

In [17]:
def getEditDistance(x,w):
    transpose=False
    matrix = np.zeros((len(x)+1, len(w)+1))  
    for i in range(len(x)+1):
        matrix[i,0]=i
    for j in range(len(w)+1):
        matrix[0,j]=j
    for i in range(0,len(x)):
        for j in range(0,len(w)):
            if x[i]==w[j]:
                matrix[i+1][j+1] = min(matrix[i+1][j] + 1, matrix[i][j+1] + 1, matrix[i][j])
            else:
                 matrix[i+1][j+1] = min(matrix[i+1][j] + 1, matrix[i][j+1] + 1, matrix[i][j] + 1)
                    
    edit=matrix[len(x),len(w)]
    if (len(x)==len(w)):
        sorted_x = sorted(x)
        sorted_w = sorted(w)
        if (sorted_x==sorted_w):
            for i in range(len(x)-1):
                if x[i]==w[i+1] and x[i+1]==w[i]:
                    transpose=True
                    edit=edit-1
                    break
            #print("transpose")
    return edit

print(getEditDistance('mza','maz')) #checking

    

1.0


## Get List Of Candidate Words with Edit Distance=1
This function returns a list of all words from the vocabulary that have an edit distance of just 1 from the given word x. The unigram created for the corpus is used for the voabulary as we only need to go through all the unique words. Otherwise if we go through the whole text, it will waste a lot of time as the total words are 63 million + where as unique words in it are around 35000 

In [18]:
def getCandidateWords(x, vocabulary):
    candidates=[]
    for v in vocabulary:
        #print(v)
        if (getEditDistance(x,v)==1):
            if v not in candidates:
                candidates.append(v)
            else:
              print("repeat")
        #print("Gotten edit")
    return candidates

print(getCandidateWords('ba', unigram)) #unigram used to only check unique words to save time

['ya', 'bat', 'bi', 'bt', 'ma', 'ka', 'ab', 'bal', 'ja', 'na', 'ga', 'ha', 'be', 'bl', 'bs', 'bad', 'da', 'aa', 'ban', 'la', 'by', 'bar', 'pa', 'bc', 'ta', 'baz', 'sa', 'bh', 'bta', 'bai', 'bna', 'bj', 'bas', 'ra', 'baj', 'mba', 'fa', 'bra', 'wa', 'bn', 'ia', 'bap', 'bk', 'bae', 'br', 'bag', 'baa', 'bia', 'bf', 'bay', 'aba', 'za', 'qa', 'bah', 'bv', 'bb', 'ua', 'oa', 'bda', 'bak', 'bo', 'ea', 'bba', 'uba', 'bla', 'bd', 'bp', 'bja', 'bua', 'oba', 'bu', 'bam', 'dba', 'bx', 'va', 'bm', 'bw', 'ca', 'bha', 'bz', 'eba', 'bau', 'iba', 'xa', 'bao', 'bg', 'boa', 'bya', 'bax']


## Find Most Probable Word
Using the functions above, first this function gets list of candidate words for x. Then for every word probability is calculated as P(x|w)*p(w) where P(x|w) is gotten using the getProbability function and P(w) using the unigram for the corpus. Then using argmax, the word with highest probability is returned.

In [30]:
def getMostProbable(x):
    if x in unigram: #the word is correct spelling, exists in vocabulary
      print("Correctly Spelled Word:", end=" ")
      return x
    else:
      candidates=getCandidateWords(x,unigram)
      if len(candidates)>0:
          probabilities=[None]*len(candidates)
          for i,w in enumerate(candidates):
              p=getProbability(x, w)*unigram[w]
              probabilities[i]=p
              #print("Probability of ", w,"is",p)
          index=np.argmax(probabilities)
          print("You Most Likely Meant:",end=" ")
          return candidates[index]
      else:
          return "No Options Available For This Word"



# Output Examples

In [24]:
print(getMostProbable('abc'))

You Most Likely Meant: ab


In [25]:
print(getMostProbable('battay'))

You Most Likely Meant: pattay


In [26]:
print(getMostProbable('batayn'))

You Most Likely Meant: bataya


In [28]:
print(getMostProbable('mujhel'))

You Most Likely Meant: mujhe


In [31]:
print(getMostProbable('nahi'))

Correctly Spelled Word: nahi


In [32]:
print(getMostProbable('xfaxsxs'))

No Options Available For This Word


In [36]:
print(getMostProbable('lool'))

You Most Likely Meant: cool
